I am going to copy some things over from Homework 4 that we should be able to use 

In [ ]:
using JuMP, Gurobi, DelimitedFiles, Random, LinearAlgebra, DataFrames, CSV, Plots, ScikitLearn, DecisionTree, MLDataUtils;

Here is what we can use if we want to do OCT

In [ ]:
df = DataFrame(CSV.read(".csv"));
cat = [];

X = df
y = df.ycol
delete!(X, [:ycol])

categorical!(X, cat)

In [ ]:
(train_X, train_y), (test_X, test_y) = IAI.split_data(:classification, X, y);

grid = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        max_depth= 3,
        minbucket = 5,
        criterion = :misclassification
    ),   
)

IAI.fit!(grid, train_X, train_y)

This is how we did CART

In [ ]:
input = csv.read(".csv");

X = input
y = input.ycol
delete!(X, [:ycol])

md_opt = 0
md_val = zeros(Float64, 1)
md_val[1] = 1.
md_max = 8
cvs = 5
for md = 1:md_max
    model = DecisionTreeClassifier(max_depth=md)
    DecisionTree.fit!(model, Matrix(X), y)
    
    val = ScikitLearn.CrossValidation.cross_val_score(model, Matrix(X), y, cv=cvs)
    
    if (sum(val[i] for i = 1:cvs) / cvs) < md_val[1]
        md_opt = md
        md_val[1] = (sum(val[i] for i = 1:cvs) / cvs)
    end
end

model = DecisionTreeClassifier(max_depth=md_opt)
fit!(model, Matrix(X), y)
y_test_predict = ScikitLearn.predict(model, Matrix(X))

val = ScikitLearn.CrossValidation.cross_val_score(model, Matrix(X), y, cv=cvs)
sum(val[i] for i = 1:cvs) / cvs

This was the function from the recitation

In [ ]:
function one_hot_encode(X, names)
    X2 = deepcopy(X)
    deletecols!(X2, Symbol.(names))
    for i in names
        vales = unique(X[i])
        for j in vales
           X2[Symbol(string(i)*"_"*string(j))] = (X[i].==j)*1
        end
    end
    return X2
end

Random forest

In [ ]:
X = one_hot_encode(df, cat)
y = X.ycol
delete!(X,[:ycol])
(X_trainvalid, y_trainvalid), (X_test, y_test) =IAI.split_data(:classification,X, y)

X_trainvalid2 = Array{Float64}(undef, size(X_trainvalid, 1), size(X_trainvalid, 2))
for i = 1:size(X_trainvalid, 1)
    for j = 1:size(X_trainvalid, 2)
        X_trainvalid2[i, j] = X_trainvalid[i, j]
    end
end

X_test2 = Array{Float64}(undef, size(X_test, 1), size(X_test, 2))
for i = 1:size(X_test, 1)
    for j = 1:size(X_test, 2)
        X_test2[i, j] = X_test[i, j]
    end
end

y_trainvalid2 = Array{Float64}(undef, size(y_trainvalid))
for i = 1:size(X_trainvalid, 1)
    y_trainvalid2[i] = y_trainvalid[i]
end

model = build_forest(y_trainvalid2, X_trainvalid2, 3, 20, 0.5, 6)

y_test_predict = apply_forest(model, X_test2)
val = 0
for r = 1:size(X_test2, 1)
    if abs(y_test_predict[r] - y_test[r]) < 0.5
        val = val + 1
    end
end
val / size(X_test, 1)

Boosted trees

In [ ]:
X = one_hot_encode(df, cat)
y = X.ycol
delete!(X,[:ycol])
(X_trainvalid, y_trainvalid), (X_test, y_test) =IAI.split_data(:classification,X, y)

regr_1 = GradientBoostingClassifier()
fit!(regr_1,  Matrix(X_trainvalid), y_trainvalid)

y_test_predict = regr_1.predict_proba(Matrix(X_test))[:,2]
val = 0
for r = 1:size(X_test2, 1)
    if abs(y_test_predict[r] - y_test[r]) < 0.5
        val = val + 1
    end
end
val / size(X_test, 1)

In the homework, we didn't really go over optimal regression trees. I am going to look into this to see what we can get. Maybe this could be used to predict temperature

This is an example from the documentation

In [ ]:
# df = DataFrame(CSV.File(
#     "yacht_hydrodynamics.data",
#     delim=' ',            # file uses ' ' as separators rather than ','
#     ignorerepeated=true,  # sometimes columns are separated by more than one ' '
#     header=[:position, :prismatic, :length_displacement, :beam_draught,
#             :length_beam, :froude, :resistance],
# ))

In [ ]:
grid = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
    ),
    max_depth=1:5,
)
IAI.fit!(grid, train_X, train_y)
IAI.get_learner(grid)

With linear predictions

In [ ]:
grid = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
        max_depth=2,
        regression_sparsity=:all,
    ),
    regression_lambda=[0.0005, 0.001, 0.005],
)
IAI.fit!(grid, train_X, train_y)
IAI.get_learner(grid)

We have continuous variables
 - Temperature
 - Precipitation
 - Pressure
 - Humidity
 - Time

We will have some parameter variables
 - The one talking about if it was clear